In [26]:
%run -i "../classes.py"
%run -i "../functions.py"

import pickle
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [27]:
model_settings_all_datasets = {
     "best_word2vec":
    {
        "model_path": "../all_datasets/word2vec_models/word2vec_vs50_win5_sg0.model",
        "vector_size": 50,
        "window_size": 5,
        "is_skipgram": False
    }
}

model_settings_clickbait_dataset = {
     "best_word2vec":
    {
        "model_path": "../all_datasets/word2vec_models/word2vec_vs20_win4_sg1.model",
        "vector_size": 20,
        "window_size": 4,
        "is_skipgram": True
    }
}

model_settings_fake_news = {
     "best_word2vec":
    {
        "model_path": "../all_datasets/word2vec_models/word2vec_vs100_win3_sg0.model",
        "vector_size": 100,
        "window_size": 3,
        "is_skipgram": False
    }
}


model_w2v_all_datasets = Word2VecModel(model_settings_all_datasets["best_word2vec"])
model_w2v_clickbait_dataset = Word2VecModel(model_settings_clickbait_dataset["best_word2vec"])
model_w2v_fake_news = Word2VecModel(model_settings_fake_news["best_word2vec"])

In [28]:
variables_to_drop_all_datasets = get_dimensions_to_drop("../all_datasets/")
variables_to_drop_clickbait_dataset_small = get_dimensions_to_drop("../clickbait_dataset/")
variables_to_drop_clickbait_dataset_large = get_dimensions_to_drop("../clickbait_dataset_more_variables/")
variables_to_drop_fake_news = get_dimensions_to_drop("../fake_news/")

In [29]:
sorted(variables_to_drop_clickbait_dataset_small)

[0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [30]:
sorted(variables_to_drop_clickbait_dataset_large)

[4, 6, 8, 9, 10, 11, 12, 13, 14, 16, 18, 19]

In [31]:
with open('../data/preprocessed_titles_labels.pkl', 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df=df[df['sample']=='test'].reset_index(drop=True)

df['title_vector_all-datasets'] = [get_word_vectors(model_w2v_all_datasets, title, aggregation='mean') for title in df['title']]
df['title_vector_clickbait-dataset-small'] = [get_word_vectors(model_w2v_clickbait_dataset, title, aggregation='mean') for title in df['title']]
df['title_vector_clickbait-dataset-large'] = [get_word_vectors(model_w2v_clickbait_dataset, title, aggregation='mean') for title in df['title']]
df['title_vector_fake-news'] = [get_word_vectors(model_w2v_fake_news, title, aggregation='mean') for title in df['title']]
df.head()

,title,is_clickbait,text,dataset,sample,title_vector_all-datasets,title_vector_clickbait-dataset-small,title_vector_clickbait-dataset-large,title_vector_fake-news
0,"[florida, back, away, prep, schedule, favoring...",0,NaN,clickbait-dataset,test,"[0.2498653, -0.8647978, -0.7974327, -1.0866896...","[0.17065616, 0.02894783, 0.40574288, 0.428069,...","[0.17065616, 0.02894783, 0.40574288, 0.428069,...","[0.51622945, 0.73041844, 0.75695777, -0.203301..."
1,"[beck, comey, letter, one, irresponsible, thin...",1,Home › POLITICS › BECK: COMEY LETTER ‘ONE OF T...,fake-news,test,"[1.541091, -0.21590804, -0.100614995, 0.210063...","[-0.6798498, -0.6573447, 0.39795816, 0.1067116...","[-0.6798498, -0.6573447, 0.39795816, 0.1067116...","[-0.054407395, -0.69897795, -1.1749345, -0.241..."
2,"[sydney, swelter, climate, change, link, scien...",0,Southeastern Australia has suffered through a ...,fake-news,test,"[-0.10031766, 0.7332149, 1.4093989, 1.3071582,...","[0.20888309, -0.11385598, 0.3459676, 0.3343268...","[0.20888309, -0.11385598, 0.3459676, 0.3343268...","[0.71581674, -1.0366164, -0.9906624, 0.3276617..."
3,"[california, today, airbnb, hometown, success,...",0,Good morning. This is the last day of our test...,fake-news,test,"[-0.4853912, 0.7275942, 1.7871649, 0.51797223,...","[0.0943965, 0.16424643, 0.6883686, 0.3321835, ...","[0.0943965, 0.16424643, 0.6883686, 0.3321835, ...","[-0.32054478, 0.103009775, -0.5056469, -0.6569..."
4,"[clinton, could, win, control, voting, machine...",1,Voter Fraud is Trump’s Greatest Challenge New ...,fake-news,test,"[-0.38561773, -1.0883421, 1.6974436, 0.8922161...","[0.022586528, -0.33844036, -0.22179739, 0.4941...","[0.022586528, -0.33844036, -0.22179739, 0.4941...","[0.9339156, -1.0556666, 0.23963484, 0.03732006..."


In [32]:
df['title_vector_all-datasets'] = [drop_dimensions_from_vector(vector, variables_to_drop_all_datasets) for vector in df['title_vector_all-datasets']]
df['title_vector_clickbait-dataset-small'] = [drop_dimensions_from_vector(vector, variables_to_drop_clickbait_dataset_small) for vector in df['title_vector_clickbait-dataset-small']]
df['title_vector_clickbait-dataset-large'] = [drop_dimensions_from_vector(vector, variables_to_drop_clickbait_dataset_large) for vector in df['title_vector_clickbait-dataset-large']]
df['title_vector_fake-news'] = [drop_dimensions_from_vector(vector, variables_to_drop_fake_news) for vector in df['title_vector_fake-news']]

df.head()

,title,is_clickbait,text,dataset,sample,title_vector_all-datasets,title_vector_clickbait-dataset-small,title_vector_clickbait-dataset-large,title_vector_fake-news
0,"[florida, back, away, prep, schedule, favoring...",0,NaN,clickbait-dataset,test,"[-0.8647978, -1.0866896, 1.5393616, -1.7940332...","[-0.15725407, 0.52002305]","[0.17065616, 0.02894783, 0.40574288, 0.428069,...","[0.51622945, -0.7859521, 0.73046845, -0.580015..."
1,"[beck, comey, letter, one, irresponsible, thin...",1,Home › POLITICS › BECK: COMEY LETTER ‘ONE OF T...,fake-news,test,"[-0.21590804, 0.21006308, 0.37189576, 0.254671...","[0.23223898, 0.12198795]","[-0.6798498, -0.6573447, 0.39795816, 0.1067116...","[-0.054407395, 0.35865083, 0.03550728, 1.21892..."
2,"[sydney, swelter, climate, change, link, scien...",0,Southeastern Australia has suffered through a ...,fake-news,test,"[0.7332149, 1.3071582, 0.5699282, -0.99958676,...","[-0.27072743, 0.834452]","[0.20888309, -0.11385598, 0.3459676, 0.3343268...","[0.71581674, -0.73093647, -0.67854905, -1.0576..."
3,"[california, today, airbnb, hometown, success,...",0,Good morning. This is the last day of our test...,fake-news,test,"[0.7275942, 0.51797223, -0.25978324, -1.720556...","[-0.097397685, 0.55998206]","[0.0943965, 0.16424643, 0.6883686, 0.3321835, ...","[-0.32054478, 0.25544903, -0.3354072, -0.32854..."
4,"[clinton, could, win, control, voting, machine...",1,Voter Fraud is Trump’s Greatest Challenge New ...,fake-news,test,"[-1.0883421, 0.8922161, 0.11775681, 0.06603074...","[-0.30219448, 0.6367484]","[0.022586528, -0.33844036, -0.22179739, 0.4941...","[0.9339156, -0.583611, -0.05104524, -0.7793181..."


In [33]:
import copy as cp
df_all_datasets = cp.deepcopy(df)
df_clickbait_dataset = df[df['dataset']=='clickbait-dataset'].reset_index(drop=True)
df_fake_news = df[df['dataset']=='fake-news'].reset_index(drop=True)

df_all_datasets.shape, df_clickbait_dataset.shape, df_fake_news.shape


((2590, 9), (1595, 9), (995, 9))

In [34]:
df_all_datasets.head()

,title,is_clickbait,text,dataset,sample,title_vector_all-datasets,title_vector_clickbait-dataset-small,title_vector_clickbait-dataset-large,title_vector_fake-news
0,"[florida, back, away, prep, schedule, favoring...",0,NaN,clickbait-dataset,test,"[-0.8647978, -1.0866896, 1.5393616, -1.7940332...","[-0.15725407, 0.52002305]","[0.17065616, 0.02894783, 0.40574288, 0.428069,...","[0.51622945, -0.7859521, 0.73046845, -0.580015..."
1,"[beck, comey, letter, one, irresponsible, thin...",1,Home › POLITICS › BECK: COMEY LETTER ‘ONE OF T...,fake-news,test,"[-0.21590804, 0.21006308, 0.37189576, 0.254671...","[0.23223898, 0.12198795]","[-0.6798498, -0.6573447, 0.39795816, 0.1067116...","[-0.054407395, 0.35865083, 0.03550728, 1.21892..."
2,"[sydney, swelter, climate, change, link, scien...",0,Southeastern Australia has suffered through a ...,fake-news,test,"[0.7332149, 1.3071582, 0.5699282, -0.99958676,...","[-0.27072743, 0.834452]","[0.20888309, -0.11385598, 0.3459676, 0.3343268...","[0.71581674, -0.73093647, -0.67854905, -1.0576..."
3,"[california, today, airbnb, hometown, success,...",0,Good morning. This is the last day of our test...,fake-news,test,"[0.7275942, 0.51797223, -0.25978324, -1.720556...","[-0.097397685, 0.55998206]","[0.0943965, 0.16424643, 0.6883686, 0.3321835, ...","[-0.32054478, 0.25544903, -0.3354072, -0.32854..."
4,"[clinton, could, win, control, voting, machine...",1,Voter Fraud is Trump’s Greatest Challenge New ...,fake-news,test,"[-1.0883421, 0.8922161, 0.11775681, 0.06603074...","[-0.30219448, 0.6367484]","[0.022586528, -0.33844036, -0.22179739, 0.4941...","[0.9339156, -0.583611, -0.05104524, -0.7793181..."


In [35]:
model_combinations = ['mod_all-datasets', 'mod_fake-news', 'mod_clickbait-dataset-small', 'mod_clickbait-dataset-large']
dataset_combinations = ['data_all_datasets', 'data_fake-news', 'data_clickbait-dataset']
metrics = ['accuracy', 'precision', 'recall', 'f1-score', 'roc-auc']
tresholds = ['defaultThresh', 'optimizedF1Thresh']

results = dict()

for model in model_combinations:
    results[model] = dict()
    for dataset in dataset_combinations:
        results[model][dataset] = dict()
        for threshold in tresholds:
            results[model][dataset][threshold] = dict()
            for metric in metrics:
                results[model][dataset][threshold][metric] = None




In [36]:
with open('../all_datasets/predictive_models/catboost.pkl', 'rb') as f:
    model_all_datasets = pickle.load(f)

with open('../clickbait_dataset/predictive_models/catboost.pkl', 'rb') as f:
    model_clickbait_dataset_small = pickle.load(f)

with open('../clickbait_dataset_more_variables/predictive_models/lightgbm.pkl', 'rb') as f:
    model_clickbait_dataset_large = pickle.load(f)

with open('../fake_news/predictive_models/catboost.pkl', 'rb') as f:
    model_fake_news = pickle.load(f)

In [37]:
with open('../all_datasets/predictive_models/threshold.txt', 'rb') as f:
    threshold_all_datasets = float(f.read())

with open('../clickbait_dataset/predictive_models/threshold.txt', 'rb') as f:
    threshold_clickbait_dataset_small = float(f.read())

with open('../clickbait_dataset_more_variables/predictive_models/threshold.txt', 'rb') as f:
    threshold_clickbait_dataset_large = float(f.read())

with open('../fake_news/predictive_models/threshold.txt', 'rb') as f:
    threshold_fake_news = float(f.read())

In [38]:
threshold_all_datasets, threshold_clickbait_dataset_small, threshold_clickbait_dataset_large, threshold_fake_news

(0.38823882388238823,
 0.32493249324932494,
 0.13731373137313732,
 0.39793979397939794)

In [39]:
with open('../all_datasets/predictive_models/scaler.pkl', 'rb') as f:
    scaler_all_datasets = pickle.load(f)

with open('../clickbait_dataset/predictive_models/scaler.pkl', 'rb') as f:
    scaler_clickbait_dataset_small = pickle.load(f)

with open('../clickbait_dataset_more_variables/predictive_models/scaler.pkl', 'rb') as f:
    scaler_clickbait_dataset_large = pickle.load(f)

with open('../fake_news/predictive_models/scaler.pkl', 'rb') as f:
    scaler_fake_news = pickle.load(f)


c:\Users\adamm\Desktop\sheeesh\pracaInzynierska\.conda\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# All datasets data

In [40]:
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
for model in tqdm(model_combinations, desc='models'):
    print(model)
    if model == 'mod_all-datasets':
        scaler = cp.deepcopy(scaler_all_datasets)
        predictor = cp.deepcopy(model_all_datasets)
        opt_threshold = threshold_all_datasets
    elif model == 'mod_fake-news':
        scaler = cp.deepcopy(scaler_fake_news)
        predictor = cp.deepcopy(model_fake_news)
        opt_threshold = threshold_fake_news
    elif model == 'mod_clickbait-dataset-small':
        scaler = cp.deepcopy(scaler_clickbait_dataset_small)
        predictor = cp.deepcopy(model_clickbait_dataset_small)
        opt_threshold = threshold_clickbait_dataset_small
    elif model == 'mod_clickbait-dataset-large':
        scaler = cp.deepcopy(scaler_clickbait_dataset_large)
        predictor = cp.deepcopy(model_clickbait_dataset_large)
        opt_threshold = threshold_clickbait_dataset_large
       

    for dataset in tqdm(dataset_combinations, desc='datasets'):
        if dataset == 'data_all_datasets':
            df_temp = cp.deepcopy(df_all_datasets)
            dataset_name = 'all_datasets'
        elif dataset == 'data_fake-news':
            df_temp = cp.deepcopy(df_fake_news)
            dataset_name = 'fake-news'
        elif dataset == 'data_clickbait-dataset':
            df_temp = cp.deepcopy(df_clickbait_dataset)
            dataset_name = 'clickbait-dataset'

        for threhold, threshold_name in zip([0.5, opt_threshold], ['defaultThresh', 'optimizedF1Thresh']):
            model_name_split = model.split('_')[1]

            X = np.vstack(df_temp['title_vector_' + model_name_split].values)
            y_true = df_temp['is_clickbait']
            X_scaled = scaler.transform(X)

            y_pred_proba = predictor.predict_proba(X_scaled)[:,1]
            y_pred = (y_pred_proba > threhold).astype(int)


            results[model][dataset][threshold_name]['accuracy'] = accuracy_score(y_true, y_pred)
            results[model][dataset][threshold_name]['precision'] = precision_score(y_true, y_pred)
            results[model][dataset][threshold_name]['recall'] = recall_score(y_true, y_pred)
            results[model][dataset][threshold_name]['f1-score'] = f1_score(y_true, y_pred)
            results[model][dataset][threshold_name]['roc-auc'] = roc_auc_score(y_true, y_pred_proba)



    


models:   0%|          | 0/4 [00:00<?, ?it/s]

mod_all-datasets


datasets:   0%|          | 0/3 [00:00<?, ?it/s]

mod_fake-news


datasets:   0%|          | 0/3 [00:00<?, ?it/s]

mod_clickbait-dataset-small


datasets:   0%|          | 0/3 [00:00<?, ?it/s]

mod_clickbait-dataset-large


datasets:   0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
results

{'mod_all-datasets': {'data_all_datasets': {'defaultThresh': {'accuracy': 0.8679536679536679,
    'precision': 0.8946474086661003,
    'recall': 0.8284815106215578,
    'f1-score': 0.8602941176470589,
    'roc-auc': 0.9433612355639807},
   'optimizedF1Thresh': {'accuracy': 0.8675675675675676,
    'precision': 0.8531202435312024,
    'recall': 0.8819826907946499,
    'f1-score': 0.8673114119922631,
    'roc-auc': 0.9433612355639807}},
  'data_fake-news': {'defaultThresh': {'accuracy': 0.7798994974874371,
    'precision': 0.8832335329341318,
    'recall': 0.6210526315789474,
    'f1-score': 0.7292954264524104,
    'roc-auc': 0.8908825910931174},
   'optimizedF1Thresh': {'accuracy': 0.807035175879397,
    'precision': 0.8409638554216867,
    'recall': 0.7347368421052631,
    'f1-score': 0.7842696629213484,
    'roc-auc': 0.8908825910931174}},
  'data_clickbait-dataset': {'defaultThresh': {'accuracy': 0.922884012539185,
    'precision': 0.8991696322657177,
    'recall': 0.9522613065326633,

In [42]:
df_clickbait_dataset

,title,is_clickbait,text,dataset,sample,title_vector_all-datasets,title_vector_clickbait-dataset-small,title_vector_clickbait-dataset-large,title_vector_fake-news
0,"[florida, back, away, prep, schedule, favoring...",0,NaN,clickbait-dataset,test,"[-0.8647978, -1.0866896, 1.5393616, -1.7940332...","[-0.15725407, 0.52002305]","[0.17065616, 0.02894783, 0.40574288, 0.428069,...","[0.51622945, -0.7859521, 0.73046845, -0.580015..."
1,"[everything, need, know, tv, movie, two, thous...",1,NaN,clickbait-dataset,test,"[1.1802026, -0.16725393, 0.8712863, 1.6209981,...","[0.28868902, 0.017361019]","[-0.05450148, -0.48198387, 0.3551623, 0.505231...","[1.0075414, -0.33693862, -0.55501586, 1.137571..."
2,"[stock, slip, despite, friday, rally]",0,NaN,clickbait-dataset,test,"[-1.0087417, -0.17675415, 0.17658488, -1.75624...","[0.35191944, 1.4064429]","[-0.1368265, 0.15313981, 0.30979222, 0.3594320...","[0.8479103, 0.1851492, -0.79149836, 0.8003437,..."
3,"[eight, team, threaten, splinter, formula, one]",0,NaN,clickbait-dataset,test,"[-0.9253955, -0.15865856, 3.0873997, 0.2408208...","[-0.62794423, 0.4982635]","[0.06719932, -0.26081622, -0.0024915815, 0.908...","[-0.09059701, -0.7047113, 0.08391233, -1.60536..."
4,"[peril, pleasure, entrepreneurship]",0,NaN,clickbait-dataset,test,"[-0.035615563, 0.5070497, 1.701961, -2.5328505...","[-0.3739367, 0.78412116]","[-0.6698522, -0.5955941, 1.5253737, 1.2993803,...","[-1.4687334, -0.48967338, -1.2960594, -1.14340..."
...,...,...,...,...,...,...,...,...,...
1590,"[turkey, set, price, lift, ban, youtube, googl...",0,NaN,clickbait-dataset,test,"[-0.4189353, 0.45532674, 0.37400407, -1.170164...","[0.027313383, 0.810994]","[0.39515972, -0.13425931, 0.1928797, 0.5171408...","[-0.81012905, -0.5532097, -0.07359877, -1.3045..."
1591,"[meredith, grey, meredith, grey]",1,NaN,clickbait-dataset,test,"[0.11928892, 0.47666153, 1.1520201, 2.3778, 0....","[-0.4203969, 0.5485832]","[-0.5101028, -0.2207199, 1.0685674, -0.4320862...","[-0.9279984, -1.5021099, -0.5609312, 0.3389249..."
1592,"[downing, street, memo, forum, held, june, 16t...",0,NaN,clickbait-dataset,test,"[0.78568804, -0.2841628, -0.4046898, 0.2296878...","[0.011007412, 0.9013174]","[-0.14751107, -0.113367274, 0.22253686, 0.4802...","[0.02414503, -0.3516948, 0.044164363, -0.37546..."
1593,"[airplane, crash, nigeria]",0,NaN,clickbait-dataset,test,"[-0.75491637, -0.8767419, 1.5620588, 1.0690575...","[-0.5687345, 1.4191499]","[0.30306473, 0.4950346, -0.52282053, -0.146760...","[0.5887533, 0.1666786, -0.042548496, 1.5828816..."


In [43]:
X = np.vstack(df_clickbait_dataset['title_vector_clickbait-dataset-large'].values)
X_scaled = scaler_clickbait_dataset_large.transform(X)
y = df_clickbait_dataset['is_clickbait']


y_pred_proba = model_clickbait_dataset_large.predict_proba(X_scaled)[:,1]
y_pred = (y_pred_proba > threshold_clickbait_dataset_large).astype(int)

f1_score(y, y_pred)

0.9931120851596743